In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

In [ ]:
import logging
import matplotlib.pyplot as plt
import torch
from pykeen.pipeline import pipeline
from statistics import mean, median
import pandas as pd
import numpy as np
import seaborn as sns
import networkx as nx
from pyvis.network import Network
from ipywidgets import interact
import ipywidgets as widgets
from itertools import chain
from pathlib import Path
from rich import print
import plwordnet
import random
from pykeen.triples import TriplesFactory
from pykeen.datasets import EagerDataset

from utils import prepare_for_visualization

## Settings

In [ ]:
import warnings

warnings.filterwarnings("ignore")
warnings.simplefilter("ignore")

pd.set_option("display.float_format", lambda x: "%.5f" % x)
logging.getLogger("pykeen").setLevel(logging.CRITICAL)

## Loading data

WordNet to semantyczna sieć słów składająca się z zestawów synonimów, zwanych synsetami, zorganizowanych w sposób hierarchiczny. Pomysł na słowosieć zrodził się na Uniwersytecie Princeton i od tego czasu stała się popularnym narzędziem w lingwistyce komputerowej.

Podstawową jednostką w słowosieci jest synset, czyli zbiór synonimów reprezentujących jeden konkretny koncept lub znaczenie słowa. Na przykład słowa "samochód" i "auto" mogą tworzyć jeden synset, ponieważ wszystkie odnoszą się do tego samego konceptu.

Relacje w słowosieci są kluczowym elementem jej struktury. Więcej o nich: http://nlp.pwr.wroc.pl/25-narzedzia-zasoby/wiedza/81-relacje-w-slowosieci 

In [ ]:
WORDNET_PATH = Path("data/plwordnet/plwordnet_4_2.xml")

In [ ]:
wn = plwordnet.load(str(WORDNET_PATH))
print(wn)

## Ćwiczenie

Przygotuj dane do treningu spełniając poniższe warunki:
1. Pozostaw synsety, w których jednostki leksylane są wyłącznie rzeczownikami pos=`NOUN`.
2. Pozostaw 150 000 losowych synsetów.
3. Pozsotaw 50 pierwszych typów relacji (sortowanie wg. id).

Podpowiedzi:
1. Końcowo zbiór powinien mieć formę macierzy Numpy (wymiar n x 3) złożonej z stringów:
    ```
    [['{rozmowa.1 konwersacja.1 dialog.1 dyskurs.1}', 'hiperonimia', '{pogawędka.1 pogaduszka.1 pogwarka.1 pogaducha.1 rozmówka.1 gawędka.1 gawęda.3 gadu-gadu.1}']]
    ```
2. Strukturę wordnetu najszybciej zrozumieć analizaując początek tego pliku: https://github.com/maxadamski/plwordnet/blob/main/plwordnet/wordnet.py

In [ ]:
def filter_pos_synsets(synsets: list, pos: str) -> list:
    raise NotImplementedError()

filtered_synsets = filter_pos_synsets(wn.synsets, "NOUN")

sampled_data = ...

assert len(sampled_data.shape) == 2
assert isinstance(sampled_data[0][0], str)

In [ ]:
triples = TriplesFactory.from_labeled_triples(sampled_data)
training, testing = triples.split(0.95)
dataset = EagerDataset(training, testing)

In [ ]:
g = nx.DiGraph()
g.add_edges_from([(h, t, {"title": r}) for h, r, t in dataset.training.triples])

## Visualization

In [ ]:
_list_nodes = list(g.nodes)


@interact
def visualize(
    nodes=widgets.SelectMultiple(
        options=list(g.nodes), rows=10, value=[_list_nodes[0]]
    ),
    k=[0, 1, 2, 3],
    toggle_physics=False,
):
    filtered = set(
        chain(
            *[
                list(nx.single_source_shortest_path_length(g, n, cutoff=k))
                for n in nodes
            ]
        )
    )
    # print(filtered)
    subgraph = nx.subgraph_view(g, filter_node=lambda x: x in filtered)
    nt = Network(
        "500px", "500px", directed=True, notebook=True, cdn_resources="in_line"
    )
    nt.inherit_edge_colors(False)
    nt.from_nx(subgraph)
    nt.toggle_physics(toggle_physics)
    display(nt.show("basic.html"))

## EDA

In [ ]:
data = []
for subset_name in ["training", "testing"]:
    subset_metrics = {"subset": subset_name}
    subset = dataset.__getattribute__(subset_name)
    triples = subset.triples
    subset_metrics["num_triples"] = len(triples)
    subset_metrics["num_entities"] = len(np.unique(triples[:, [0, 2]]))
    subset_metrics["num_relations"] = len(np.unique(triples[:, 1]))
    data.append(subset_metrics)

pd.DataFrame(data)

In [ ]:
metrics = {}
metrics["n_connected_components"] = nx.number_connected_components(g.to_undirected())
metrics["mean_size_of_connected_components"] = mean(
    len(c) for c in nx.connected_components(g.to_undirected())
)
metrics["median_size_of_connected_components"] = median(
    len(c) for c in nx.connected_components(g.to_undirected())
)
metrics["density"] = nx.density(g)
metrics["number_of_selfloops"] = nx.number_of_selfloops(g)
metrics["average_clustering"] = nx.average_clustering(g)
pd.DataFrame({"training": metrics})

In [ ]:
degree_sequence = sorted(g.degree(), key=lambda x: x[1], reverse=True)[:20]

x, y = zip(*degree_sequence)
fig, axes = plt.subplots(1, 2, figsize=(15, 5))
axes[0].set_title("Degree of nodes")
axes[0].barh(y=x, width=y)
axes[1].set_title("Degree Histogram")
sns.histplot([d for n, d in g.degree()], ax=axes[1], log_scale=True)
plt.tight_layout()

## Training 

In [ ]:
TRAIN_MODELS = True # or load pretrained

In [ ]:
save_location = Path("results/plwordnet")

if TRAIN_MODELS:
    result = pipeline(
        dataset=dataset,
        model="TransE",
        model_kwargs={"embedding_dim": 32},
        loss="nssa",
        loss_kwargs={"adversarial_temperature": 0.34, "margin": 9},
        optimizer="Adam",
        optimizer_kwargs={"lr": 0.004},
        negative_sampler_kwargs={"num_negs_per_pos": 33},
        training_kwargs=dict(
            num_epochs=25,
            batch_size=512,
            use_tqdm_batch=False,
        ),
        random_seed=123,
    )
    save_location.mkdir(exist_ok=True, parents=True)
    result.save_to_directory(save_location)
    print(f"Saved: {os.listdir(save_location)}")

## Metrics

In [ ]:
if TRAIN_MODELS:
    result.plot_losses()
    plt.show()

In [ ]:
if TRAIN_MODELS:
    metrics = result.metric_results.to_df()
    display(metrics[(metrics.Side == "both") & (metrics.Type == "realistic")])

## Embeddings visualization

In [ ]:
if TRAIN_MODELS:
    model = result.model
else:
    model  = torch.load(save_location / 'trained_model.pkl')

In [ ]:
embeddings = model.entity_representations[0](torch.arange(dataset.num_entities))
labels = np.array(
    [dataset.training.entity_id_to_label[i] for i in range(dataset.num_entities)]
)

In [ ]:
sampled_embeddings_idx = random.sample(list(range(len(embeddings))), 5000)

In [ ]:
prepare_for_visualization(
    embeddings.detach().numpy()[sampled_embeddings_idx],
    labels[sampled_embeddings_idx],
    Path("logs/plwordnet"),
)

In [ ]:
!tensorboard --logdir=logs/plwordnet

## Prediction

Przeklej kod z poprzedniego notebooka.